In [42]:
import pandas as pd
import numpy as np

import locale
locale.setlocale(locale.LC_ALL,'')

'Spanish_Mexico.1252'

In [43]:
df = pd.read_excel("data.xlsx",'data')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1516 entries, 0 to 1515
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Número de movimiento      1516 non-null   int64  
 1   Documento                 1516 non-null   object 
 2   Clave de artículo         1516 non-null   int64  
 3   Descripción               1516 non-null   object 
 4   Línea                     1516 non-null   object 
 5   Clave cliente/proveedor   1516 non-null   object 
 6   Concepto                  1516 non-null   int64  
 7   Fecha                     1516 non-null   object 
 8   Vendedor                  1494 non-null   float64
 9   Precio                    1516 non-null   float64
 10  Costo                     1516 non-null   float64
 11  Almacén                   1516 non-null   int64  
 12  Tipo de documento         1516 non-null   object 
 13  Costo operado             1516 non-null   float64
 14  Cantidad

In [44]:
df["Clave de artículo "] = df["Clave de artículo "].astype(object)
df["Fecha "] = pd.to_datetime(df["Fecha "], dayfirst=True)
df["Cantidad "] = pd.to_numeric(df["Cantidad "])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1516 entries, 0 to 1515
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Número de movimiento      1516 non-null   int64         
 1   Documento                 1516 non-null   object        
 2   Clave de artículo         1516 non-null   object        
 3   Descripción               1516 non-null   object        
 4   Línea                     1516 non-null   object        
 5   Clave cliente/proveedor   1516 non-null   object        
 6   Concepto                  1516 non-null   int64         
 7   Fecha                     1516 non-null   datetime64[ns]
 8   Vendedor                  1494 non-null   float64       
 9   Precio                    1516 non-null   float64       
 10  Costo                     1516 non-null   float64       
 11  Almacén                   1516 non-null   int64         
 12  Tipo de documento   

In [45]:
dfA = df[df["Almacén "] == 1][["Fecha ","Cantidad "]]

dfprov = dfA.groupby("Fecha ").sum()
dfprov["Cantidad "] = dfprov["Cantidad "] *-1
dfprov["Dias "] = 1
dfprov["Probabilidad "] = 1 / dfprov["Cantidad "].count()

tabla = dfprov.groupby("Cantidad ").agg({"Dias ":'count',"Probabilidad ":'sum'})
tabla["Si-dias "] = tabla["Dias "].cumsum()
tabla["No-dias "] = tabla["Dias "].sum() - tabla["Si-dias "]
tabla["Si-% "] = tabla["Si-dias "] / tabla["Dias "].sum()
tabla["No-% "] = tabla["No-dias "] / tabla["Dias "].sum()

tabla

,Dias,Probabilidad,Si-dias,No-dias,Si-%,No-%
Cantidad,,,,,,
-4,1,0.003106,1,321,0.003106,0.996894
-3,1,0.003106,2,320,0.006211,0.993789
-1,3,0.009317,5,317,0.015528,0.984472
0,4,0.012422,9,313,0.027950,0.972050
1,67,0.208075,76,246,0.236025,0.763975
2,62,0.192547,138,184,0.428571,0.571429
3,52,0.161491,190,132,0.590062,0.409938
4,35,0.108696,225,97,0.698758,0.301242
5,22,0.068323,247,75,0.767081,0.232919


In [46]:
tabla[tabla["Si-% "] > .95].iloc[0:1]

,Dias,Probabilidad,Si-dias,No-dias,Si-%,No-%
Cantidad,,,,,,
16,3,0.009317,307,15,0.953416,0.046584


In [47]:
almacenes = [1,2]

writer = pd.ExcelWriter("stock-mínimo.xlsx", engine="openpyxl")
    
for almacen in almacenes:

    dfA = df[df["Almacén "] == almacen][["Fecha ","Cantidad "]]

    dfprov = dfA.groupby("Fecha ").sum()
    dfprov["Cantidad "] = dfprov["Cantidad "] *-1
    dfprov["Dias "] = 1
    dfprov["Probabilidad "] = 1 / dfprov["Cantidad "].count()

    tabla = dfprov.groupby("Cantidad ").agg({"Dias ":'count',"Probabilidad ":'sum'})
    tabla["Si-dias "] = tabla["Dias "].cumsum()
    tabla["No-dias "] = tabla["Dias "].sum() - tabla["Si-dias "]
    tabla["Si-% "] = tabla["Si-dias "] / tabla["Dias "].sum()
    tabla["No-% "] = tabla["No-dias "] / tabla["Dias "].sum()
    
    tabla.to_excel(writer, sheet_name=f"A{almacen}")

writer.close()
writer.handles = None